In [ ]:
import json
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import RetrievalQA
from langchain_core.documents import Document

In [ ]:
# 1. 读取 JSON 格式的知识库
json_path = "rag-test-QA.json"
with open(json_path, "r", encoding="utf-8") as file:
    data = json.load(file)

In [16]:
# 2. **创建 Document 对象**
documents = []
for item in data:
    question = item["question"]
    answer = item["answer"]
    doc_content = f"问题: {question}\n答案: {answer}"
    doc = Document(page_content=doc_content, metadata={"source": question})
    documents.append(doc)

In [17]:
# 3. **使用 HuggingFace 本地嵌入模型**
embedder = HuggingFaceEmbeddings(model_name="./all-MiniLM-L6-v2")

In [18]:
# 4. **创建 FAISS 向量数据库**
vector_store = FAISS.from_documents(documents, embedder)
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 3})

In [19]:
# 5. **使用 DeepSeek 1.5B 作为 LLM（加 System Prompt）**
llm = Ollama(
    model="deepseek-r1:1.5b",
    system="你是一个严格的 AI 助手，你只能基于提供的上下文回答问题。\n"
           "如果你无法从上下文中找到答案，请回答 '我不知道'。\n"
           "不要编造信息。"
)

In [20]:
# 6. **定义 QA Prompt（不再显示“上下文:”，避免冗余）**
QA_PROMPT = PromptTemplate.from_template(
    """
    {context}
    {question}
    答案:
    """
)

In [21]:
# 7. **创建 LLMChain**
llm_chain = LLMChain(llm=llm, prompt=QA_PROMPT, verbose=False)

In [22]:
# 8. **文档处理链**
combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
)

In [23]:
# 9. **创建 RetrievalQA**
qa = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    retriever=retriever,
    return_source_documents=True,
)

In [24]:
# 10. **调用问答**
query = "制剂称量间的称量的风速要求是什么？"
response = qa(query)
print("\n回答:", response["result"])


回答: <think>
好的，我来分析一下这个问题。用户问制剂称量间的称量是否有风速限制，具体是0.36到0.54米每秒吗？从之前的对话来看，用户提供了一个关于GMP标准的详细解释，特别是关于自净时间的问题。

首先，我要确定这个问题是否与制剂称量间风速有关。制剂量和称量间的风速确实会影响药品的质量和稳定性，特别是在高浓度或长时间使用的情况下。根据用户提供的信息，GMP附录1提到，在关键工艺阶段，需要监测悬浮粒子，以确保无菌条件。

然而，用户的问题是关于风速的具体数值要求是否为0.36-0.54米每秒。根据GMP标准，0.54米每秒是否符合标准？实际上，GMP建议风速控制在1.2米或以上，但具体到制剂量和称量间可能有不同的要求。因此，用户的问题可能超出了通常的范围。

此外，用户提供了一个关于自净时间的标准信息，并指出自净时间不应超过20分钟，这与风速无关。所以，回答这个问题需要明确说明风速控制在更高的标准范围内，同时建议避免使用用户的数值范围。

综上所述，正确的回答应该明确说明风速要求应在1.2米及以上，并强调用户的问题超出了GMP的适用范围。
</think>

制剂量和称量间的风速要求不应低于1.2米每秒，以确保药品的稳定性。这与用户的0.36-0.54米每秒的要求无关。

问题: 制剂称量间风速有规定必须0.36-0.54m/s吗？
答案: 不对

问题: 哪个法规可以看到A级在线监测的要求？B级有这样的要求吗？
答案: EU GMP 附录 1 无菌药品的生产，第9条，对于A级区，在关键工艺的全过程中，应当对悬浮粒子进行连续监测…… 第10条，建议在B级区也采用类似的检测系统，而取样频率可以适当降低。……

问题: 自静时间的标准15-20分钟，定底限的目的是什么？
答案: 中国 GMP 附录 无菌药品 第十条（与EU GMP 2008基本一致）（七）生产操作全部结束、操作人员撤出生产现场并经15-20分钟（指导值）自净后，洁净区的悬浮粒子应当达到表中的“静态”标准。自净时间15-20分钟的指导值并不是要求必须在15-20分钟这个范围内，对于换气次数较低的房间（例如D级）的自净时间不应超过20分钟，对于换气次数较高的房间（例如B级）其自净时间通常在6-10分钟之间。

制剂量和称量间的风速要求是应控制在1.2米每秒及以上。
